In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
import numpy as np
import xgboost as xgb
import pandas as pd
import matplotlib.pyplot as plt
import pickle

In [ ]:
df = pd.read_csv("lpg_test_data.csv", skiprows=0)
df.head()

In [ ]:
df2 = pd.read_csv("lpg_trained_data.csv", skiprows=0)
df2.head()

In [ ]:
out = df.loc[~df.departure_port.isin(df2.departure_port)]
out

In [ ]:
sspd = df.loc[~df.sspd_mstd.isin(df2.sspd_mstd)]
sspd

In [ ]:
df_remove = pd.concat([out, sspd]).sort_values('voyage_id')\
    .drop_duplicates().reset_index(drop=True)

In [ ]:
df_remove

In [ ]:
df_test = (df[~df.voyage_id.isin(df_remove.voyage_id)]).reset_index(drop=True)

In [ ]:
df_test

In [ ]:
X = df_test.copy()
X

In [ ]:
X.drop(['voyage_id', 'imo','mstd_id'], axis=1, inplace=True)

In [ ]:
y = X['arrival_port'].copy()
X.drop(['arrival_port'], axis=1, inplace=True)

In [ ]:
X.replace("autumn", 
           "autum", 
           inplace=True)

In [ ]:
file = open("dp_enc.obj",'rb')
dp_enc = pickle.load(file)
file.close()
X["departure_port"] = dp_enc.transform(X["departure_port"])
############
file = open("sea_enc.obj",'rb')
sea_enc = pickle.load(file)
file.close()
X["season"] = sea_enc.transform(X["season"])
##############
file = open("seg_enc.obj",'rb')
seg_enc = pickle.load(file)
file.close()
X["sub_segment"] = seg_enc.transform(X["sub_segment"])
#######################
file = open("sspd_enc.obj",'rb')
sspd_enc = pickle.load(file)
file.close()
X["sspd_mstd"] = sspd_enc.transform(X["sspd_mstd"])

In [ ]:
loaded_model = pickle.load(open("lpg_model.dat", "rb"))
y_pred = loaded_model.predict_proba(X) 

In [ ]:
file = open("dp_enc.obj",'rb')
dp_enc = pickle.load(file)
file.close()
X["departure_port"] = dp_enc.inverse_transform(X["departure_port"])
############
file = open("sea_enc.obj",'rb')
sea_enc = pickle.load(file)
file.close()
X["season"] = sea_enc.inverse_transform(X["season"])
##############
file = open("seg_enc.obj",'rb')
seg_enc = pickle.load(file)
file.close()
X["sub_segment"] = seg_enc.inverse_transform(X["sub_segment"])
#######################
file = open("sspd_enc.obj",'rb')
sspd_enc = pickle.load(file)
file.close()
X["sspd_mstd"] = sspd_enc.inverse_transform(X["sspd_mstd"])

In [ ]:
# loaded_model.classes_
file = open("enc_targets.obj",'rb')
dp_enc = pickle.load(file)
file.close()
label_classes = dp_enc.inverse_transform(loaded_model.classes_)

In [ ]:
pred_To_Class_map = pd.DataFrame(y_pred*100, columns=label_classes)
pred_To_Class_map

In [ ]:
ports, prob = [pred_To_Class_map.idxmax(axis=1), pred_To_Class_map.max(axis=1)]
ports[0]

In [ ]:
pred_To_Class_map['JPNGO']

In [ ]:
prob[0]

In [ ]:
df_predicted = pd.DataFrame({'predicted_ports': ports, 'predicted_probability': prob}, columns=['predicted_ports', 'predicted_probability'])

In [ ]:
df_predicted

In [ ]:
new_predicted_df = pd.concat([df_test, df_predicted], axis = 1)

In [ ]:
new_predicted_df.drop(columns=['imo','sub_segment','departure_port','trajectory_length','sspd_mstd','sspd_dist','mstd_id','probablity','distance_ratio','season'], inplace=True)

In [ ]:
new_predicted_df.drop(columns=['arrival_port'],inplace=True)

In [ ]:
lpg_predicted_data = df.merge(new_predicted_df, how='left', on='voyage_id' )


In [ ]:
lpg_predicted_data

In [ ]:
lpg_predicted_data.predicted_ports.fillna(lpg_predicted_data.sspd_mstd, inplace=True)


In [ ]:
lpg_predicted_data.predicted_probability.fillna((lpg_predicted_data.probablity)*100, inplace=True)

In [ ]:
lpg_predicted_data

In [ ]:
lpg_predicted_data.to_csv(r'chemical_predicted.csv', index=False)

In [ ]:
comparison_column = np.where(lpg_predicted_data["arrival_port"] == lpg_predicted_data["predicted_ports"], True, False)

In [ ]:
sum(comparison_column)